In [11]:
import requests
import time
import zipfile
import io
import pandas as pd


def export_surveys(survey_id_dict):
    surveys = {}
    # Replace these with your actual credentials
    API_TOKEN = '8d0zWNKj6WnY0kLjmoecUtwgacbwXBLgQmPT0PWZ'
    DATA_CENTER = 'yul1'  # e.g., 'ca1', 'eu1', 'us2', etc.

    for survey_name in survey_id_dict:
        print(f"Exporting {survey_name}...")
        SURVEY_ID = survey_id_dict[survey_name]['SURVEY_ID']

        # Set up headers and URL
        BASE_URL = f"https://{DATA_CENTER}.qualtrics.com/API/v3/surveys/{SURVEY_ID}/export-responses/"
        HEADERS = {
            "Content-Type": "application/json",
            "X-API-TOKEN": API_TOKEN
        }
        # Start the export
        response = requests.post(BASE_URL, headers=HEADERS, json={"format": "csv"})
        print("Export response:", response.json())
        progress_id = response.json()["result"]["progressId"]

        # Wait for the export to be complete
        while True:
            check_url = BASE_URL + progress_id
            check_response = requests.get(check_url, headers=HEADERS)
            status = check_response.json()["result"]["status"]
            if status == "complete":
                file_id = check_response.json()["result"]["fileId"]
                break
            elif status == "failed":
                raise Exception("Export failed")
            time.sleep(2)

        # Download the file
        download_url = BASE_URL + f"{file_id}/file"
        download_response = requests.get(download_url, headers=HEADERS, stream=True)

        # Unzip and read the CSV into a DataFrame
        zip_file = zipfile.ZipFile(io.BytesIO(download_response.content))
        csv_file = zip_file.open(zip_file.namelist()[0])
        surveys[survey_name] = pd.read_csv(csv_file)

    return surveys

# Find survey ID by the URL for the survey you want
# It will be something like https://rutgers.yul1.qualtrics.com/jfe/form/SV_0pQR9T8rGz1A2jk
# where SV_0pQR9T8rGz1A2jk is the SURVEY_ID

survey_id_dict = {
    "PCX_ClinicalVisit_ClinicianRatings":{
        "SURVEY_ID": "SV_bpUc7WXekERE7Cm"
    },
    "PCX_ClinicalVisit_SelfReport":{
        "SURVEY_ID": "SV_78QRYTSOnegCSjQ"
    },
    "PCX_fMRIVisit_SelfReport":{
        "SURVEY_ID": "SV_0UqGfGjgsl2nklU"
    },
    "PCX_SupplementalBattery":{
        "SURVEY_ID": "SV_08nF8tsZ4NU0rWe"
    },
}


surveys = export_surveys(survey_id_dict)



Exporting PCX_ClinicalVisit_ClinicianRatings...
Export response: {'result': {'progressId': 'ES_43lGKMKEMlgp0k6', 'percentComplete': 0.0, 'status': 'inProgress'}, 'meta': {'requestId': 'db6f8958-d862-43a7-a4e7-ca3e775ae8a3', 'httpStatus': '200 - OK'}}
Exporting PCX_ClinicalVisit_SelfReport...
Export response: {'result': {'progressId': 'ES_9uJb433ujTj3Dg2', 'percentComplete': 0.0, 'status': 'inProgress'}, 'meta': {'requestId': 'a4377eff-31de-4eac-844c-8a5c172ab0cf', 'httpStatus': '200 - OK'}}
Exporting PCX_fMRIVisit_SelfReport...
Export response: {'result': {'progressId': 'ES_7ONSe1mbOnki2Ym', 'percentComplete': 0.0, 'status': 'inProgress'}, 'meta': {'requestId': '02411fbe-d06a-4572-9597-c785cce5a991', 'httpStatus': '200 - OK'}}
Exporting PCX_SupplementalBattery...
Export response: {'result': {'progressId': 'ES_0Stcfw5mYTfPebs', 'percentComplete': 0.0, 'status': 'inProgress'}, 'meta': {'requestId': 'ef90b77b-0989-4c5d-ab7a-cf8a93438a8a', 'httpStatus': '200 - OK'}}


In [12]:
df = surveys['PCX_SupplementalBattery']
rsri_cols = [col for col in df.columns if 'rsri_' in col]
df[['SUBJECT_ID']+rsri_cols]


,SUBJECT_ID,rsri_time_First Click,rsri_time_Last Click,rsri_time_Page Submit,rsri_time_Click Count,rsri_1,rsri_2,rsri_3,rsri_4,rsri_5,...,rsri_21,rsri_22,rsri_23,rsri_24,rsri_25,rsri_26,rsri_27,rsri_28,rsri_29,rsri_30
0,Subject ID (PCXXXX),Timing - First Click,Timing - Last Click,Timing - Page Submit,Timing - Click Count,"On average, how often per year were you absent...","On average, how often per year were you sent t...",Did you have illnesses/symptoms such as headac...,How often do you have nightmares?,Were you scared of the dark?,...,"During recess, did you play with the main grou...",Did you enjoy participating in party games?,Did you enjoy meeting new children your age?,"Did your voice squeak, crack, or sound shaky w...",How popular did you feel?,"Did you have any problems with, or have to see...",Did you need a night-light or hall light on in...,Did you willingly participate in group singing...,Were your feelings easily hurt?,Did you tell your friends or family members wh...
1,"{""ImportId"":""QID100_TEXT""}","{""ImportId"":""QID13_FIRST_CLICK""}","{""ImportId"":""QID13_LAST_CLICK""}","{""ImportId"":""QID13_PAGE_SUBMIT""}","{""ImportId"":""QID13_CLICK_COUNT""}","{""ImportId"":""QID18""}","{""ImportId"":""QID22""}","{""ImportId"":""QID31""}","{""ImportId"":""QID23""}","{""ImportId"":""QID24""}",...,"{""ImportId"":""QID42""}","{""ImportId"":""QID43""}","{""ImportId"":""QID44""}","{""ImportId"":""QID45""}","{""ImportId"":""QID46""}","{""ImportId"":""QID47""}","{""ImportId"":""QID48""}","{""ImportId"":""QID49""}","{""ImportId"":""QID50""}","{""ImportId"":""QID51""}"
2,123,13.421,13.421,16.782,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gravida sem nullam sem a ante. Amet sollicitud...,0.92,9.31,2.54,15,5,2,4,3,999,...,999,3,2,4,999,1,1,3,5,3
4,Nibh nonummy. Blandit nibh augue. Porttitor la...,1.95,5.89,5.25,36,4,4,1,4,5,...,2,3,5,999,2,5,5,4,2,1
5,Sem sollicitudin dui turpis pellentesque magni...,2.57,3.54,9.92,50,2,4,1,3,5,...,4,5,3,2,3,2,2,999,5,3
6,Turpis felis quis sem! Integer praesent mattis...,2.9,5.29,8.65,0,5,5,999,999,3,...,1,1,4,5,4,5,4,2,1,5
7,"Tempus tempora, a viverra blandit tempus sem d...",1.4,8.74,7.66,44,2,3,4,3,999,...,5,1,1,2,5,1,5,3,1,1
8,123,2.064,34.432,35.182,35,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,test1025,1.014,172.027,172.833,35,3,3,3,3,3,...,2,2,2,2,2,2,2,2,2,2
